In [6]:
# Import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import os

# Disable OneDNN to avoid performance issues
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

# Set up data generators for Arthritis dataset
arthritis_train_datagen = ImageDataGenerator(rescale=1/255)
arthritis_validation_datagen = ImageDataGenerator(rescale=1/255)

# Train dataset
arthritis_train_dataset = arthritis_train_datagen.flow_from_directory(
    '/workspaces/ATAi_advance_notebooks/health-app-backend/arthritis_data/train', 
    target_size=(400, 400), 
    batch_size=32, 
    class_mode='binary'
)

# Validation dataset
arthritis_validation_dataset = arthritis_validation_datagen.flow_from_directory(
    '/workspaces/ATAi_advance_notebooks/health-app-backend/arthritis_data/validate', 
    target_size=(400, 400), 
    batch_size=32, 
    class_mode='binary'
)

# Define the Arthritis detection model
arthritis_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(400, 400, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
arthritis_model.compile(loss='binary_crossentropy',
                        optimizer=RMSprop(lr=0.01),
                        metrics=['accuracy'])

# Train the model
history = arthritis_model.fit(
    arthritis_train_dataset, 
    steps_per_epoch=50, 
    epochs=5, 
    validation_data=arthritis_validation_dataset
)

# Plot training accuracy and loss
def plot_training(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label="Training Accuracy")
    plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()
    plt.plot(epochs, loss, 'r', label="Training Loss")
    plt.plot(epochs, val_loss, 'b', label="Validation Loss")
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

# Visualize training results
plot_training(history)

# Example: Making predictions on a new arthritis image
def predict_image(image_path, model, target_size=(400, 400)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Create batch axis
    img_array /= 255.0  # Normalize image to [0,1]

    prediction = model.predict(img_array)

    if prediction > 0.5:
        print(f'Prediction: Arthritis detected with {prediction[0][0]*100:.2f}% confidence')
    else:
        print(f'Prediction: No arthritis with {(1 - prediction[0][0])*100:.2f}% confidence')

# Test the model with a sample image
predict_image('/workspaces/ATAi_advance_notebooks/health-app-backend/arthritis_data/test/sample1.jpg', arthritis_model)


ModuleNotFoundError: No module named 'numpy.typing'